In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load the decrypted file into a pandas DataFrame
df = pd.read_excel("/content/USM FY 2014 New FT and Transfers Retention Degree and FA  12-21-23 (1) (1)(1).xlsx")

# Display the first few ows
print(df.head())

    campusid   opeid  FirstTimeStatus  Race_New  AttendStatus  Gender AppZip  \
0  @00147352  210400                5         1             2       2  20705   
1  @00159764  210400                5         3             1       2  20876   
2  @00170784  210400                5         9             2       2  21211   
3  @00186283  210400                5         5             2       2  20852   
4  @00191705  210400                5         5             1       2  20794   

   EntryMajor  EntryDegree  FY2014Fall  ...  FirstTimeFlag  \
0      120300           40      2014.0  ...              5   
1      121300           40      2014.0  ...              5   
2      120300           40      2014.0  ...              5   
3      120300           40      2014.0  ...              5   
4      120300           40      2014.0  ...              5   

           FirstTimeFlagDesc  Race_NewID      Race_Short  GenderID  \
0  Undergraduate Transfer-In           1           Black         2   
1  Und

In [ ]:
# Function to determine if any degree date columns are not null (indicating graduation)
def check_graduation(row):
    # Check each degree date column for non-null values
    for i in range(1, 5):  # Assuming you have degreedate1 to degreedate4
        if pd.notnull(row[f'degreedate{i}']):
            return 'Yes'
    return 'No'

#Function to see if the degree obtained was in STEM
hegisstem = ('7','9','4','17','19');

myctr = 0;
def check_stem(row):
  #ctr = 0;
  if (str(row['EntryMajor']).startswith(hegisstem)):
    #print("found 17");
    return 'Yes';
  #print();
  return 'No';


# Apply the function to create the "Graduated" column
print('Total Students: ', len(df.index));
df['Graduated'] = df.apply(check_graduation, axis=1);
print('Total Graduated: ', len(df[df['Graduated'] == 'Yes']))
df['Stem1'] = df.apply(check_stem, axis=1);
#df['stem2stem'] = df.apply(lambda x: 1 if (x['Stem1'] == 'Yes') and (x['HegisDeg1'] == 'Yes') else 0, axis=1)
print('Stem Majors: ', (df.Stem1.values == 'Yes').sum());
print ('Stem Majors Graduated: ', len(df[(df['Stem1'] == 'Yes') & (df['Graduated'] == 'Yes')]))
#print (len(df[(df['Graduated'] == 'Yes']) && (df['Stem1'] == 'Yes')))

# Save the updated DataFrame to a new CSV file
df.to_csv('updated_file.csv', index=False);
#print(df.head(5));




Total Students:  30606
Total Graduated:  17645
Stem Majors:  14151
Stem Majors Graduated:  7774


In [ ]:
# Identify rows where entrydegreeseeking is 'Non-Degree undergrad'
rows_to_drop = df[df['entrydegreeseeking'] == 'Non-Degree undergrad'].index

# Drop these rows from the DataFrame
df.drop(rows_to_drop, inplace=True)
print('After we drop non degree seeking students')

# Now df contains only the rows where entrydegreeseeking is not 'Non-Degree undergrad'
# You can continue your analysis using df without saving it to a new CSV file

# Calculate total support across specified years
# df['totalsupport'] = df.loc[:, 'GrantAmountFY2014':'MeritAmountFY2022'].sum(axis=1)
if all(col in df.columns for col in ['GrantAmountFY2014', 'MeritAmountFY2022']):
   df['totalsupport'] = df.loc[:, 'GrantAmountFY2014':'MeritAmountFY2022'].sum(axis=1)
else:
   print("Required columns for totalsupport calculation are missing.")

# Indicate if supported in FY2014
df['supportedY1'] = df.apply(lambda x: 1 if (x['GrantAmountFY2014'] + x['MeritAmountFY2014'] != 0) else 0, axis=1)

# Indicate if supported at all based on 'totalsupport'
df['supported'] = df.apply(lambda x: 1 if (x['totalsupport'] != 0) else 0, axis=1)

# Indicate if graduated
df['grdnum'] = df.apply(lambda x: 1 if (x['Graduated'] == 'Yes') else 0, axis=1)
print('Number of Students: ', len(df.index));
print('Number of Stem Students: ', (df.Stem1.values == 'Yes').sum());
#print(df.head(5));
#print('StemGradRates')
print ('STEM Graduates: ', len(df[(df['Stem1'] == 'Yes') & (df['Graduated'] == 'Yes')]))


After we drop non degree seeking students
Number of Students:  28960
Number of Stem Students:  12509
STEM Graduates:  7660


In [ ]:
# Function to categorize EFC values, including handling nulls
def categorize_efc(efc):
    if pd.isnull(efc):
        return 'Unknown'  # Handle null values explicitly
    elif efc < 6657 :
        return 'High'
    elif 6657 <= efc <= 13312 :
        return 'Medium'
    else:  # EFC > 13312
        return 'Low'

# Apply the function to the EFC column to create a new category column
df['EFC_Category'] = df['EFC'].apply(categorize_efc)
print(df.head(5));




    campusid   opeid  FirstTimeStatus  Race_New  AttendStatus  Gender AppZip  \
0  @00147352  210400                5         1             2       2  20705   
1  @00159764  210400                5         3             1       2  20876   
2  @00170784  210400                5         9             2       2  21211   
3  @00186283  210400                5         5             2       2  20852   
4  @00191705  210400                5         5             1       2  20794   

   EntryMajor  EntryDegree  FY2014Fall  ...  Status_Desc  degree  \
0      120300           40      2014.0  ...    Part-Time      40   
1      121300           40      2014.0  ...    Full-Time      40   
2      120300           40      2014.0  ...    Part-Time      40   
3      120300           40      2014.0  ...    Part-Time      40   
4      120300           40      2014.0  ...    Full-Time      40   

   entrydegreeseeking  Graduated  Stem1  totalsupport  supportedY1  supported  \
0          Bachelor's        

In [ ]:
efc_category_counts = df["EFC_Category"].value_counts()
print(efc_category_counts)

EFC_Category
High       9771
Unknown    9047
Low        7579
Medium     2563
Name: count, dtype: int64


In [ ]:
# Ensure totalsupport is in a numeric format, in case it's not
df['totalsupport'] = pd.to_numeric(df['totalsupport'], errors='coerce')

#check the shape of the data
print(df.shape)

# Drop rows where totalsupport is NaN after conversion
df.dropna(subset=['totalsupport'], inplace=True)
print(len(df.index))

# Define bins for support levels
bins = [0,1, 5000, 10000, 15000, 20000, np.inf]
labels = ['NoSup','<5K', '5K-10K', '10K-15K', '15K-20K', '>20K']
df['support_bin'] = pd.cut(df['totalsupport'], bins=bins, labels=labels, right=False)
#print(df.head(5));
# Count the number of students in each bin
student_counts = df['support_bin'].value_counts().sort_index()

# Print the results
print('Students in different support bins:\n', student_counts)

(28960, 107)
28960
Students in different support bins:
 support_bin
NoSup      12096
<5K         4514
5K-10K      2794
10K-15K     1888
15K-20K     1547
>20K        6121
Name: count, dtype: int64


In [ ]:
# Filter students who started and ended in STEM
stem_to_stem_df = df[
    (df['EntryMajor'].astype(str).str.startswith(tuple(hegisstem))) &
    (df['HegisDeg1'].astype(str).str.startswith(tuple(hegisstem))) &
    (df['Graduated'] == 'Yes')
]
stem_allstem_df = df[
    (df['HegisDeg1'].astype(str).str.startswith(tuple(hegisstem))) &
    (df['Graduated'] == 'Yes')
]
stem_graduates = len(df[(df['Stem1'] == 'Yes') & (df['Graduated'] == 'Yes')])


# Count calculations
total_students = len(df)  # Total students in dataset
total_stem_students = (df['Stem1'] == 'Yes').sum()  # Total students who entered in STEM
stem_to_stem_count = len(stem_to_stem_df)  # Students who started & ended in STEM
#stem_graduates = len(df[(df['Stem1'] == 'Yes') & (df['Graduated'] == 'Yes')])  # STEM students who graduated
stem_all=len(stem_allstem_df )

# Percentage calculations
percentage_of_total_students = (stem_to_stem_count / total_students) * 100
percentage_of_stem_students = (stem_to_stem_count / total_stem_students) * 100
percentage_of_stem_students_grad = (stem_to_stem_count / stem_graduates) * 100
percentage_of_stem_students_grad_all = (stem_all / total_students) * 100


# Display results
print(f"Total Students: {total_students}")
print(f"Total STEM Entry Students: {total_stem_students}")
print(f"Total STEM grad all: {stem_all}")
print(f"Students Who Started and Ended in STEM: {stem_to_stem_count}")
print(f"Percentage of Total Students: {percentage_of_total_students:.2f}%")
print(f"Percentage of STEM Entry Students: {percentage_of_stem_students:.2f}%")
print(f"Percentage of STEM grad_all: {percentage_of_stem_students_grad_all:.2f}%")

#print(f"Percentage of STEM Graduates (7660): {percentage_of_stem_students_grad:.2f}%")


Total Students: 28960
Total STEM Entry Students: 12509
Total STEM grad all: 5485
Students Who Started and Ended in STEM: 4923
Percentage of Total Students: 17.00%
Percentage of STEM Entry Students: 39.36%
Percentage of STEM grad_all: 18.94%


In [ ]:
# Recalculating the EFC breakdown to ensure consistency

# Group by EFC category for all students and STEM entry students
efc_total_students = df.groupby('EFC_Category').size()
efc_stem_entrants = df[df['Stem1'] == 'Yes'].groupby('EFC_Category').size()
efc_stem_completed = stem_to_stem_df.groupby('EFC_Category').size()
efc_stem_entrants_who_graduated = df[(df['Stem1'] == 'Yes') & (df['Graduated'] == 'Yes')].groupby('EFC_Category').size()

# Perform parallel calculations

# STEM-to-STEM % of STEM Entrants Calculation
parallel_stem_completion_rate_from_entrants = (efc_stem_completed / efc_stem_entrants) * 100

# STEM-to-STEM % of Total Students Calculation
parallel_stem_completion_rate_from_total = (efc_stem_completed / efc_total_students) * 100

# STEM Entrants Graduation Rate from Total Calculation
parallel_stem_entrants_graduation_rate = (efc_stem_entrants_who_graduated / efc_total_students) * 100
parallel_stem_entrants_graduation_rate2 = (efc_stem_entrants_who_graduated / efc_stem_entrants) * 100


# Create a DataFrame for parallel calculations
parallel_calculations_df = pd.DataFrame({
        'Total_Students': efc_total_students,

    'STEM Entrants': efc_stem_entrants,
    'STEM-To-STEM Completed': efc_stem_completed,
        'STEM_Entrants_Who_Graduated(Any Major)': efc_stem_entrants_who_graduated,

    'STEM Entrants Graduation Rate from stem entry': parallel_stem_entrants_graduation_rate2,

    'STEM-To-STEM % of STEM Entrants': parallel_stem_completion_rate_from_entrants,
    'STEM-To-STEM % of Total Students': parallel_stem_completion_rate_from_total,
    'STEM Entrants Graduation Rate from Total': parallel_stem_entrants_graduation_rate
}).fillna(0)  # Fill NaN values with 0

parallel_calculations_df


,Total_Students,STEM Entrants,STEM-To-STEM Completed,STEM_Entrants_Who_Graduated(Any Major),STEM Entrants Graduation Rate from stem entry,STEM-To-STEM % of STEM Entrants,STEM-To-STEM % of Total Students,STEM Entrants Graduation Rate from Total
EFC_Category,,,,,,,,
High,9771,3714,1320,2195,59.100700,35.541195,13.509364,22.464436
Low,7579,3568,1808,2660,74.551570,50.672646,23.855390,35.096978
Medium,2563,1059,458,735,69.405099,43.248347,17.869684,28.677331
Unknown,9047,4168,1337,2070,49.664107,32.077735,14.778380,22.880513


In [ ]:
# Combine both calculations into a single DataFrame

# Calculate STEM Completion Rate from STEM Entrants
stem_completion_rate_from_entrants = (efc_support_stem_completed / efc_support_stem_entrants) * 100

# Calculate STEM Completion Rate from Total Students
stem_completion_rate_from_total = (efc_support_stem_completed / efc_support_total_students) * 100

# Calculate STEM Entry Rate from Total Students
stem_entry_rate_from_total = (efc_support_stem_entrants / efc_support_total_students) * 100

# Create a combined DataFrame for parallel calculations
combined_parallel_calculations_df = pd.DataFrame({
    'Total_Students': efc_support_total_students,
    'STEM_Entrants': efc_support_stem_entrants,
    'STEM_To_Stem_Completed': efc_support_stem_completed,
    'STEM_Entry_Rate_From_Total': stem_entry_rate_from_total,
    'STEM_To_Stem_Completion_Rate_From_Entrants': stem_completion_rate_from_entrants,
    'STEM_To_Stem_Completion_Rate_From_Total': stem_completion_rate_from_total
}).fillna(0)  # Fill NaN values with 0

combined_parallel_calculations_df


Total_Students  STEM_Entrants  STEM_To_Stem_Completed  \
EFC_Category supported                                                          
High         0                     272             87                      12   
             1                    9499           3627                    1308   
Low          0                    2112            804                     274   
             1                    5467           2764                    1534   
Medium       0                     665            238                      62   
             1                    1898            821                     396   
Unknown      0                    9047           4168                    1337   

                        STEM_Entry_Rate_From_Total  \
EFC_Category supported                               
High         0                           31.985294   
             1                           38.182967   
Low          0                           38.068182   
             1                           50.557893   
Medium       0                           35.789474   
             1                           43.256059   
Unknown      0                           46.070521   

                        STEM_To_Stem_Completion_Rate_From_Entrants  \
EFC_Category supported                                               
High         0                                           13.793103   
             1                                           36.062862   
Low          0                                           34.079602   
             1                                           55.499276   
Medium       0                                           26.050420   
             1                                           48.233861   
Unknown      0                                           32.077735   

                        STEM_To_Stem_Completion_Rate_From_Total  
EFC_Category supported                                           
High         0                                         4.411765  
             1                                        13.769871  
Low          0                                        12.973485  
             1                                        28.059265  
Medium       0                                         9.323308  
             1                                        20.864067  
Unknown      0                                        14.778380

In [ ]:
# Extracting values for calculations

# For Supported (1) group
supported_stem_to_stem_count = parallel_calculations_df.loc[(['High', 'Low', 'Medium'], 1), 'STEM-to-STEM Count'].sum()
supported_stem_entry_count = efc_support_stem_entry_count.loc[(['High', 'Low', 'Medium'], 1)].sum()
supported_stem_total_count = parallel_calculations_df.loc[(['High', 'Low', 'Medium'], 1), 'STEM TOTAL Count'].sum()

# For Not Supported (0) group
not_supported_stem_to_stem_count = parallel_calculations_df.loc[(['High', 'Low', 'Medium'], 0), 'STEM-to-STEM Count'].sum()
not_supported_stem_entry_count = efc_support_stem_entry_count.loc[(['High', 'Low', 'Medium'], 0)].sum()
not_supported_stem_total_count = parallel_calculations_df.loc[(['High', 'Low', 'Medium'], 0), 'STEM TOTAL Count'].sum()

# STEM-to-STEM Count / STEM Entry Count for each group
supported_stem_to_stem_ratio = (supported_stem_to_stem_count / supported_stem_entry_count) * 100
not_supported_stem_to_stem_ratio = (not_supported_stem_to_stem_count / not_supported_stem_entry_count) * 100

# STEM-to-STEM Count / STEM TOTAL Count for each group
supported_stem_to_stem_total_ratio = (supported_stem_to_stem_count / supported_stem_total_count) * 100
not_supported_stem_to_stem_total_ratio = (not_supported_stem_to_stem_count / not_supported_stem_total_count) * 100

# Print the results clearly
print("===== STEM-to-STEM Ratios =====")
print(f"STEM-to-STEM % of STEM Entry (Supported): {supported_stem_to_stem_ratio:.2f}%")
print(f"STEM-to-STEM % of STEM Entry (Not Supported): {not_supported_stem_to_stem_ratio:.2f}%")

print("\n===== STEM-to-STEM Ratios (Total STEM Students) =====")
print(f"STEM-to-STEM % of Total STEM (Supported): {supported_stem_to_stem_total_ratio:.2f}%")
print(f"STEM-to-STEM % of Total STEM (Not Supported): {not_supported_stem_to_stem_total_ratio:.2f}%")


===== STEM-to-STEM Ratios =====
STEM-to-STEM % of STEM Entry (Supported): 44.90%
STEM-to-STEM % of STEM Entry (Not Supported): 30.82%

===== STEM-to-STEM Ratios (Total STEM Students) =====
STEM-to-STEM % of Total STEM (Supported): 19.20%
STEM-to-STEM % of Total STEM (Not Supported): 11.41%
